In [ ]:
from deep_orderbook.recorder import Receiver
from deep_orderbook.shapper import BookShapper

import pandas as pd
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETHBTC", "BTCUSDT", "ETHUSDT", "BNBBTC", "BNBETH", "BNBUSDT"]
receiver = await Receiver.create(markets=MARKETS)

In [ ]:
#receiver2 = await dob.recorder.Writer.create(markets=['BNBBTC'], data_folder='data')

In [ ]:
b = receiver.depth_managers['ETHBTC'].get_depth_cache().get_bids()
pd.DataFrame(b, columns=['price', 'size']).astype(np.float64).set_index('price')#.cumsum().plot()

In [ ]:
a = receiver.depth_managers['ETHBTC'].get_depth_cache().get_asks()
pd.DataFrame(a, columns=['price', 'size']).astype(np.float64).set_index('price')

In [ ]:
receiver.depth_managers['ETHBTC'].trades

In [ ]:
shappers = {pair: await BookShapper.create() for pair in MARKETS}
multi_replay = receiver.multi_generator(shappers)

In [ ]:
_ = await multi_replay.__anext__()

In [ ]:
import aioitertools
genarr = BookShapper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await aioitertools.next(genarr)

In [ ]:
genacc = BookShapper.accumulate_array(genarr, markets=MARKETS)
_ = await aioitertools.next(genacc)

In [ ]:
every = 1
LENGTH = 512
x = []
async for n,sec in aioitertools.enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(25,16))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower");
        clear_output(wait=True)
        plt.show()
    pass